In [ ]:
import math
import os
import shutil

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tqdm import tqdm

In [ ]:
batch_size = 32
img_height = 180
img_width = 180

In [ ]:
# Remove folder that contains less than X sample:
MIN_PHOTO = 10
d = {}
for folder in os.listdir("lfw"):
    d[folder] = len(os.listdir(os.path.join("lfw", folder)))
for key in list(d.keys()):
    if d[key] < MIN_PHOTO:
        shutil.rmtree(os.path.join("lfw", key))

In [ ]:
basedir = "lfw"
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    basedir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size,
)

In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    basedir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size,
)

In [ ]:
class_names = train_ds.class_names
print(class_names[0:10])

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
resize_and_rescale = tf.keras.Sequential(
    [
        layers.experimental.preprocessing.Resizing(img_height, img_width),
        layers.experimental.preprocessing.Rescaling(1.0 / 255),
    ]
)
data_augmentation = tf.keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
        layers.experimental.preprocessing.RandomRotation(0.2),
    ]
)

In [ ]:
model = tf.keras.Sequential(
    [
        resize_and_rescale,
        data_augmentation,
        layers.Conv2D(128, 3, activation="relu"),
        layers.MaxPooling2D(),
        layers.Conv2D(128, 3, activation="relu"),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, activation="relu"),
        layers.MaxPooling2D(),
        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dense(len(class_names), activation="softmax"),
    ]
)
model.compile(
    optimizer="adam",
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

In [ ]:
model.fit(train_ds, validation_data=val_ds, epochs=20, batch_size=batch_size)

In [ ]:
loss, acc = model.evaluate(val_ds)
print("Accuracy", acc)

In [ ]:
for images, labels in val_ds.take(10):
    #     fig = plt.figure(figsize=(5, 5))
    pred_prob = [(np.argmax(p), max(p)) for p in model.predict(images)]
    predictions = [class_names[a[0]] for a in pred_prob]
    expected = [class_names[a] for a in labels]
    not_match = [i for i in range(len(predictions)) if predictions[i] != expected[i]]
    rows = columns = int(math.ceil(len(not_match)))
    for i in range(len(not_match)):
        #         fig.add_subplot(rows, columns, i + 1)
        plt.imshow(images[not_match[i]].numpy().astype("uint8"))
        plt.title("{}|{}".format(expected[not_match[i]], predictions[not_match[i]]))
        #         plt.axis("off")
        #         plt.imshow(img)
        plt.show()

# Predict using face embeddings

In [ ]:
# load our serialized face detector from disk
print("[INFO] loading face detector...")
protoPath = "dnn_models/deploy.prototxt"
modelPath = "dnn_models/res10_300x300_ssd_iter_140000.caffemodel"
detector = cv2.dnn.readNetFromCaffe(protoPath, modelPath)
detector.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
detector.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
# load our serialized face embedding model from disk
print("[INFO] loading face recognizer...")
embedder = cv2.dnn.readNetFromTorch("dnn_models/openface.nn4.small2.v1.t7")
embedder.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
embedder.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

In [ ]:
x_train_path = []
y_train = []
x_test_path = []
y_test = []

for person in os.listdir(basedir):
    person_dir = os.path.join(basedir, person)
    person_photos = os.listdir(person_dir)
    for photo in person_photos[: int(len(person_photos) * 0.8)]:
        x_train_path.append(os.path.join(person_dir, photo))
        y_train.append(person)
    for photo in person_photos[int(len(person_photos) * 0.8) :]:
        x_test_path.append(os.path.join(person_dir, photo))
        y_test.append(person)

In [ ]:
face_path

In [1]:
X_train = []
for face_path in tqdm(x_train_path):
    image = cv2.imread(face_path)
    (h, w) = image.shape[:2]
    imageBlob = cv2.dnn.blobFromImage(image)
    detector.setInput(imageBlob)
    detections = detector.forward()
    # ensure at least one face was found
    assert len(detections) > 0
    # we're making the assumption that each image has only ONE
    # face, so find the bounding box with the largest probability
    i = np.argmax(detections[0, 0, :, 2])
    confidence = detections[0, 0, i, 2]
    # ensure that the detection with the largest probability also
    # means our minimum probability test (thus helping filter out
    # weak detections)
    if confidence > 0.5:
        # compute the (x, y)-coordinates of the bounding box for
        # the face
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")
        # extract the face ROI and grab the ROI dimensions
        face = image[startY:endY, startX:endX]
        (fH, fW) = face.shape[:2]
        # ensure the face width and height are sufficiently large
        if fW > 20 and fH > 20:
            # construct a blob for the face ROI, then pass the blob
            # through our face embedding model to obtain the 128-d
            # quantification of the face
            faceBlob = cv2.dnn.blobFromImage(
                face, 1.0, (96, 96), (0, 0, 0), swapRB=True, crop=False
            )
            embedder.setInput(faceBlob)
            vec = embedder.forward()
            # add the name of the person + corresponding face
            # embedding to their respective lists
            X_train.append(vec.flatten())

NameError: name 'tqdm' is not defined